In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, ValidationError
from typing import Optional
import pandas as pd
import re
import time
import openai
import os

In [5]:
import pandas as pd

survey_result = pd.read_csv("survey_motivasi_lari_anak_muda.csv")

examples = []

for _, row in survey_result.iterrows():
    example_dict = {
        "Nama": row["Nama"],
        "Usia": row["Usia"],
        "Jenis_Kelamin": row["Jenis Kelamin"],
        "Domisili": row["Domisili"],
        "Frekuensi_Lari_per_Minggu": row["Frekuensi Lari per Minggu"],
        "Motivasi_Utama_Lari": row["Motivasi Utama Lari"],
        "Tergabung_Komunitas": row["Tergabung Komunitas"],
        "Target_Memotivasi": row["Target Memotivasi (1-5)"],
        "Teman_Komunitas_Memotivasi": row["Teman/Komunitas Memotivasi (1-5)"],
        "Lari_Meningkatkan_Percaya_Diri": row["Lari Meningkatkan Percaya Diri (1-5)"],
        "Sering_Kehilangan_Motivasi": row["Sering Kehilangan Motivasi (1-5)"],
        "Hambatan_Utama": row["Hambatan Utama (1-5)"],
        "Hal_Menyenangkan_dari_Lari": row.get("Hal Menyenangkan dari Lari", ""),
        "Tantangan_Terbesar_Rutin_Lari": row.get("Tantangan Terbesar Rutin Lari", ""),
        "Saran_Kegiatan_Komunitas": row.get("Saran Kegiatan Komunitas", "")
    }

    formatted_string = ', '.join([f"{k}: {v}" for k, v in example_dict.items()])
    examples.append({"example": formatted_string})


In [6]:
class MotivasiLariSurvey(BaseModel):
    Nama: str
    Usia: int
    Jenis_Kelamin: str
    Domisili: str
    Frekuensi_Lari_per_Minggu: str
    Motivasi_Utama_Lari: str
    Tergabung_Komunitas: str
    Target_Memotivasi: int
    Teman_Komunitas_Memotivasi: int
    Lari_Meningkatkan_Percaya_Diri: int
    Sering_Kehilangan_Motivasi: int
    Hambatan_Utama: int
    Hal_Menyenangkan_dari_Lari: Optional[str] = None
    Tantangan_Terbesar_Rutin_Lari: Optional[str] = None
    Saran_Kegiatan_Komunitas: Optional[str] = None


In [12]:
SYNTHETIC_FEW_SHOT_PREFIX = """Berikut adalah beberapa entri survei tentang motivasi lari anak muda. Gunakan contoh ini untuk membuat entri baru yang unik dan sesuai dengan format yang diberikan."""

SYNTHETIC_FEW_SHOT_SUFFIX = """buatlah satu entri survei baru berdasarkan contoh berikut:
Subjek: {subject}
Detail tambahan: {extra}

Pastikan:
- Nama: Nama responden
- Usia: Usia responden (dalam tahun)
- Jenis Kelamin: Jenis kelamin responden (Laki-laki/Perempuan)
- Domisili: Lokasi tempat tinggal responden
- Frekuensi Lari per Minggu: Seberapa sering responden berlari dalam seminggu (misalnya: 1-2 kali, 3-4 kali, setiap hari)
- Motivasi Utama Lari: Alasan utama responden berlari (misalnya: kesehatan, kebugaran, sosial, kompetisi)
- Tergabung Komunitas: Apakah responden tergabung dalam komunitas lari (Ya/Tidak)
- Target Memotivasi: Seberapa besar target memotivasi responden untuk berlari (1-5, di mana 1 = tidak memotivasi, 5 = sangat memotivasi)
- Teman/Komunitas Memotivasi: Seberapa besar teman atau komunitas memotivasi responden untuk berlari (1-5, di mana 1 = tidak memotivasi, 5 = sangat memotivasi)
- Lari Meningkatkan Percaya Diri: Seberapa besar lari meningkatkan rasa percaya diri responden (1-5, di mana 1 = tidak meningkatkan, 5 = sangat meningkatkan)
- Sering Kehilangan Motivasi: Seberapa sering responden kehilangan motivasi untuk berlari (1-5, di mana 1 = tidak pernah, 5 = sangat sering)
- Hambatan Utama: Hambatan utama yang dihadapi responden dalam berlari (1-5, di mana 1 = tidak ada hambatan, 5 = sangat besar hambatan) 
"""

OPENAI_TEMPLATE = PromptTemplate(
    input_variables=["example"], 
    template="{example}"
    )

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

subject = "Buat entri survei baru tentang motivasi lari anak muda."
extra = "Berikan informasi yang unik dan sesuai dengan format yang telah ditentukan."


In [ ]:
llm = ChatOpenAI(
  model = "qwen/qwen3-8b:free",
  temperature = 0.7,
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("QWEN_API_KEY"),
)

In [16]:
synthetic_data = []
for i in range(10):
    prompt = prompt_template.format(subject=subject, extra=extra)
    response = llm.predict(prompt)

    print(f"\n[iterazione {i+1}] Output di dati sintetici:\n{response}")

    fields = [field.strip() for field in response.strip().split(",")]
    entry_dict = {}
    for field in fields:
        if ": " in field:
            key, value = field.split(": ", 1)
            entry_dict[key.strip()] = value.strip()

    try:
        validated = MotivasiLariSurvey(**entry_dict)
        synthetic_data.append(validated.model_dump())
    except ValidationError as e:
        print(f"[ERROR] Validation error on entry {i+1}:\n{e}")
    
    time.sleep(1.5)

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day.


RateLimitError: Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day